In [1]:
!pwd

/glade/u/home/abukowski/SAMI3-GITM-python


In [2]:
from dask_jobqueue import PBSCluster

import xarray as xr
from utility_programs.filters import make_fits

cluster = PBSCluster()

In [3]:
cluster.job_script()

'#!/usr/bin/env bash\n\n#PBS -N dask-worker\n#PBS -q regular\n#PBS -A UTAD0001\n#PBS -l select=1:ncpus=36:mem=109GB\n#PBS -l walltime=01:00:00\n\n/glade/work/abukowski/conda-envs/dask_usage/bin/python -m distributed.cli.dask_worker tcp://10.148.0.3:34519 --nthreads 6 --nworkers 6 --memory-limit 16.92GiB --name dummy-name --nanny --death-timeout 60 --interface ib0\n'

In [4]:
# cluster.adapt(minimum_jobs=0, maximum_jobs=10)

In [5]:
cluster.scale(jobs=10)

In [6]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.3:34519,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
from dask.distributed import Client
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.3:34519,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/abukowski/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:

ds = xr.open_mfdataset('/glade/u/home/abukowski/scratch/simstorm-20110521/FullStorm/GITM_2011*.nc',
                       parallel=True, engine='h5netcdf')
                      #concat_dim='time', combine='by_coords')

# ds = client.scatter(ds)



In [9]:
ds

<xarray.Dataset>
Dimensions:                          (time: 625, lon: 90, lat: 180, alt: 50)
Coordinates:
  * time                             (time) datetime64[ns] 2011-05-20 ... 201...
  * lon                              (lon) float64 2.0 6.0 10.0 ... 354.0 358.0
  * lat                              (lat) float64 -89.5 -88.5 ... 88.5 89.5
  * alt                              (alt) float64 100.0 101.7 ... 729.0 751.5
Data variables: (12/49)
    LocalTime                        (time, lon, lat) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    SolarZenithAngle                 (time, lon, lat) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    VerticalTEC                      (time, lon, lat) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    AltIntJouleHeating_W_m2          (time, lon, lat) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    AltIntHeatingTransfer_W_m2       (time, lon, lat) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    AltIntEuvHeating_W_m2            (time, lon, lat) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    ...                               ...
    e_                               (time, lon, lat, alt) float64 dask.array<chunksize=(1, 90, 180, 50), meta=np.ndarray>
    eTemperature                     (time, lon, lat, alt) float64 dask.array<chunksize=(1, 90, 180, 50), meta=np.ndarray>
    iTemperature                     (time, lon, lat, alt) float64 dask.array<chunksize=(1, 90, 180, 50), meta=np.ndarray>
    Vi_east                          (time, lon, lat, alt) float64 dask.array<chunksize=(1, 90, 180, 50), meta=np.ndarray>
    Vi_north                         (time, lon, lat, alt) float64 dask.array<chunksize=(1, 90, 180, 50), meta=np.ndarray>
    Vi_up                            (time, lon, lat, alt) float64 dask.array<chunksize=(1, 90, 180, 50), meta=np.ndarray>
Attributes:
    version:              21.14
    dropped-ghost-cells:  False
    with_time:            True

In [10]:
ds = ds.chunk({'lat':9,'lon':5,'alt':25, 'time':625})

In [11]:
ds.Rho

<xarray.DataArray 'Rho' (time: 625, lon: 90, lat: 180, alt: 50)>
dask.array<rechunk-merge, shape=(625, 90, 180, 50), dtype=float64, chunksize=(625, 5, 9, 25), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2011-05-20 ... 2011-05-22T04:00:00
  * lon      (lon) float64 2.0 6.0 10.0 14.0 18.0 ... 346.0 350.0 354.0 358.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * alt      (alt) float64 100.0 101.7 103.5 105.4 ... 684.6 706.7 729.0 751.5

In [12]:
ds2 = xr.apply_ufunc(make_fits, ds,  dask='parallelized', output_dtypes='float')

In [13]:
ds3 = 100*(ds-ds2)/ds

In [14]:
ds3.Rho.isel(time=0)

<xarray.DataArray 'Rho' (lon: 90, lat: 180, alt: 50)>
dask.array<getitem, shape=(90, 180, 50), dtype=float64, chunksize=(5, 9, 25), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] 2011-05-20
  * lon      (lon) float64 2.0 6.0 10.0 14.0 18.0 ... 346.0 350.0 354.0 358.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * alt      (alt) float64 100.0 101.7 103.5 105.4 ... 684.6 706.7 729.0 751.5

In [15]:
ds3.Temperature

<xarray.DataArray 'Temperature' (time: 625, lon: 90, lat: 180, alt: 50)>
dask.array<truediv, shape=(625, 90, 180, 50), dtype=float64, chunksize=(625, 5, 9, 25), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2011-05-20 ... 2011-05-22T04:00:00
  * lon      (lon) float64 2.0 6.0 10.0 14.0 18.0 ... 346.0 350.0 354.0 358.0
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * alt      (alt) float64 100.0 101.7 103.5 105.4 ... 684.6 706.7 729.0 751.5

In [17]:
ds3.to_netcdf('/glade/u/home/abukowski/scratch/simstorm-20110521/FullStorm/GITM_bandpass_filter.nc',
             engine='h5netcdf').compute()

/glade/scratch/abukowski/ipykernel_49079/2787475128.py:1: SerializationWarning: saving variable time with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds3.to_netcdf('/glade/u/home/abukowski/scratch/simstorm-20110521/FullStorm/GITM_bandpass_filter.nc',
/glade/work/abukowski/conda-envs/dask_usage/lib/python3.11/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 28.99 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


KilledWorker: Attempted to run task ('make_fits-rechunk-merge-rechunk-split-rechunk-merge-truediv-store-map-98532aab99651559e0cfb1f957251f39', 0, 6, 12) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://10.148.10.43:38567. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [ ]:
for t in range(len(ds3.time.values)):
    ds3.isel(time=t).to_netcdf(
        '/glade/u/home/abukowski/scratch/simstorm-20110521/FullStorm/GITM_bandpass_filter-%i.nc' % t,
        engine='h5netcdf').compute()